# Setup stuff: Connectivity 

In [ ]:
import os

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")

In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)

In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)

In [ ]:
#  CREATE GRAPH

my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)

In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())


# Setup 2 DataFrames, Nodes and Relationships ..

In [ ]:
import pandas as pd

print("--")

In [ ]:

#  Vertices/Nodes, Airports ..


df_users      = pd.read_csv('gs://farrell-bucket/mongodb/21_users.txt')

#  Frist line works, second line fails; pick one, not both ..
#
# df_movies     = pd.read_csv('gs://farrell-bucket/mongodb/E1_DoesNotError_SF10.txt')
df_movies     = pd.read_csv('gs://farrell-bucket/mongodb/E2_DoesError_SF10.txt')


df_comment_on = pd.read_csv('gs://farrell-bucket/mongodb/30_Edge_Commented.txt')

display(df_users.head(     2))
display(df_flights.head(   2))
display(df_comment_on.head(2))


In [ ]:

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_users,
      id_column             = "email",
      id_space              = "Users",
      label                 = "Users"
      )
    
   df_importer.nodes_dataframe(df_flights, 
      id_column             = "_id",
      id_space              = "Movies",
      label                 = "Movies"
      )
   
   df_importer.edges_dataframe(df_comment_on,
      source_id_space       = "Users",
      destination_id_space  = "Movies",
      source_column         = "email",
      destination_column    = "movie_id",
      type                  = "COMMENT_ON"
      )


In [ ]:

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   // LIMIT 10
   
   """, contextualize=True)

l_result.view()
